In [ ]:
import os
import arcpy
import pandas as pd
import numpy as np
import csv
from arcgis.features import GeoAccessor, GeoSeriesAccessor

arcpy.env.overwriteOutput = True

<h4>Carga de .csv con la información complementaria de los metadatos</h4>
<p>Puede complementarse con escalas y demás datos cuando se tenga<p>

In [ ]:
df_otros_metadatos_insumo = pd.read_csv("otros_metadatos.csv", sep = ";")

<h4>Filtro por capa estructurada según proceso de conversion GeoJSON - Feature Class</h4>

In [ ]:
# El proceso de conversión a geojson ejecutado incorpora datos dentro del dataframe
df_otros_metadatos_insumo_filtro = df_otros_metadatos_insumo[df_otros_metadatos_insumo['nombre_capa'].str.slice(0, 1) == '_']

<h4>Construcción de la Tabla Fuente</h4>

In [ ]:
# Se parametrizan solo los campos necesarios
df_fuente_insumo = df_otros_metadatos_insumo_filtro.loc[:,'fuente']
df_fuente_insumo = pd.DataFrame(df_fuente_insumo)

# Se agrupan todas las fuentes registradas en el DF
df_grupo_fuentes = df_fuente_insumo.groupby(['fuente']).agg({'fuente':'count'})

# Se cambia el nombre de la columna -fuente-
df_grupo_fuentes.rename({'fuente': 'conteo_fuente'}, axis = 1, inplace = True)

# Reseteo índices
df_grupo_fuentes = df_grupo_fuentes.reset_index()

# Se cambia el nombre de la columna -fuente-
df_grupo_fuentes.rename({'fuente': 'nombre_fuente'}, axis = 1, inplace = True)

# Generación del campo incremental de acuerdo al número de capas
RID = []

# Se suma uno por el header dentro del data frame
for id_rid in range(1, df_grupo_fuentes.shape[0]+1):
    RID.append(id_rid)
 
# Se utiliza el método .loc para incorporar la columna, que en este caso fue la lista que se creo previamente   
df_grupo_fuentes.loc[:,'codigo_fuente'] = RID

# Parametrizo columnas finales del DF
df_fuentes = df_grupo_fuentes[['codigo_fuente','nombre_fuente']]

# Cambio del tipo de dato
df_fuentes['codigo_fuente'].astype('double')

# Ingreso registro para capas sin fuente
df_fuentes = df_fuentes.append({'codigo_fuente': 0, 'nombre_fuente':'Sin informacion'}, ignore_index=True)

<h4>Esportada de DF a Feature Table</h4>

In [ ]:
nombre_tabla_fuentes = 'tbl_fuentes'
ruta_bd = r"C:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\Local_BD_PGAR.gdb"
ruta_tabla_fuentes = os.path.join(ruta_bd, nombre_tabla_fuentes)
df_fuentes.spatial.to_table(location=ruta_tabla_fuentes)
print("------------------------------ Se crea la tabla -tbl_fuentes- ------------------------------")